**Aufgabe 2**

In [4]:
from webencodings.mklabels import assert_lower


def lgt(v, w):
    """Find the longest common subsequence and return the score and the
    backtracking matrix.

    Return (s[n][m], b).
    """

    # Sequence 1
    l_v = [""] + list(v)

    # Sequence 2
    l_w = [""] + list(w)

    n = len(l_v)
    m = len(l_w)

    # Initialize score matrix
    s = [[0 for x in range(m)] for y in range(n)]

    # Initialize backtracking matrix
    b = [["" for x in range(m)] for y in range(n)]
    for i in range(1,n):
        b[i][0] = "oben"
    for j in range(1,m):
        b[0][j] = "links"

    for i in range(1, n):
        for j in range(1, m):
            # to distinguish between a match/mismatch.
            match = False
            if l_v[i] == l_w[j]:
                s[i][j] = max(s[i - 1][j], s[i][j - 1], s[i - 1][j - 1] + 1)
                match = True
            else:
                s[i][j] = max(s[i - 1][j], s[i][j - 1])
            
            if s[i][j] == s[i - 1][j - 1] + 1 and match:
                b[i][j] = "oben-links"
            elif s[i][j] == s[i - 1][j]:
                b[i][j] = "oben"
            elif s[i][j] == s[i][j - 1]:
                b[i][j] = "links"

    return s[n-1][m-1], b


def print_matrix(matrix, title=None):
    if title:
        print(title)
    for line in matrix:
        print(line)
    print()


# Sequences
v = "ATGCG"
w = "ATCCGC"

# Get score s_nm and backtracking matrix B
score_nm, B = lgt(v, w)

# print score s[n][m]
print("Score:", score_nm)
print()

# print backtracking matrix
print_matrix(B, title='Backtracking b:')

Score: 4

Backtracking b:
['', 'links', 'links', 'links', 'links', 'links', 'links']
['oben', 'oben-links', 'links', 'links', 'links', 'links', 'links']
['oben', 'oben', 'oben-links', 'links', 'links', 'links', 'links']
['oben', 'oben', 'oben', 'oben', 'oben', 'oben-links', 'links']
['oben', 'oben', 'oben', 'oben-links', 'oben-links', 'oben', 'oben-links']
['oben', 'oben', 'oben', 'oben', 'oben', 'oben-links', 'oben']



# Aufgabe 1

In [34]:
def backtrack_v(B, v, i, j):
    if i == 0 and j == 0:
        return ""
    if i == 0:  
        return "-" + backtrack_v(B, v, i, j - 1)
    if j == 0:  
        return v[i - 1] + backtrack_v(B, v, i - 1, j)
    if B[i][j] == "oben-links":  
        return backtrack_v(B, v, i - 1, j - 1) + v[i - 1]
    elif B[i][j] == "oben":  
        return backtrack_v(B, v, i - 1, j) + v[i - 1]
    else:  
        return backtrack_v(B, v, i, j - 1) + "-"

In [50]:
assert backtrack_v(B, v, len(v), len(w)) == "AT--GCG"

In [36]:
def backtrack_w(B, w, i, j):
    if i == 0 or j == 0:
        return ""
    if B[i][j] == "oben-links":
        return backtrack_w(B, w, i - 1, j - 1) + w[j - 1]
    elif B[i][j] == "oben":
        return backtrack_w(B, w, i - 1, j) + "-"
    else:  # "links"
        return backtrack_w(B, w, i, j - 1) + w[j - 1]

In [48]:

assert backtrack_w(B, w, len(v), len(w)) == 'ATCCGC-'


# Aufgabe 2

In [51]:
def global_alignment(v, w, match_score=1, mismatch_score=0, gap_penalty=-1):
    
    # Sequence 1
    l_v = [""] + list(v)

    # Sequence 2
    l_w = [""] + list(w)

    n = len(l_v)
    m = len(l_w)

    # Initialize score matrix with gap penalties
    s = [[0 for x in range(m)] for y in range(n)]
    B = [["" for x in range(m)] for y in range(n)]

    for i in range(1, n):
        s[i][0] = gap_penalty * i
        B[i][0] = "oben" 
    for j in range(1, m):
        s[0][j] = gap_penalty * j
        B[0][j] = "links" 

    # Fill the DP matrix based on match, mismatch, and gap penalties
    for i in range(1, n):
        for j in range(1, m):
            if l_v[i] == l_w[j]:  # Match
                score_diag = s[i - 1][j - 1] + match_score
            else:  # Mismatch
                score_diag = s[i - 1][j - 1] + mismatch_score

            score_up = s[i - 1][j] + gap_penalty  
            score_left = s[i][j - 1] + gap_penalty

            # Take the maximum of the three possible movements
            s[i][j] = max(score_diag, score_up, score_left)

            if s[i][j] == score_diag:
                B[i][j] = "oben-links"  
            elif s[i][j] == score_up:
                B[i][j] = "oben"  
            else:
                B[i][j] = "links"  

    return s[n-1][m-1], B

In [57]:
def backtrack_global_alignment(B, v, w, i, j):
    aligned_v = []
    aligned_w = []
    
    while i > 0 or j > 0:
        if i > 0 and j > 0 and B[i][j] == "oben-links":
            aligned_v.append(v[i - 1])
            aligned_w.append(w[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and (j == 0 or B[i][j] == "oben"):
            aligned_v.append(v[i - 1])
            aligned_w.append("-")
            i -= 1
        else:
            aligned_v.append("-")
            aligned_w.append(w[j - 1])
            j -= 1

    # Reverse the alignments
    return "".join(reversed(aligned_v)), "".join(reversed(aligned_w))


In [53]:
# Get score and backtracking matrix B for the global_alignment impl.
score_global, B_global = global_alignment(v, w, match_score=1, mismatch_score=0, gap_penalty=-1)

# print score s[n][m]
print("Score:", score_global)
print()

# print backtracking matrix
print_matrix(B_global, title='Backtracking B:')

Score: 3

Backtracking B:
['', 'links', 'links', 'links', 'links', 'links', 'links']
['oben', 'oben-links', 'links', 'links', 'links', 'links', 'links']
['oben', 'oben', 'oben-links', 'links', 'links', 'links', 'links']
['oben', 'oben', 'oben', 'oben-links', 'oben-links', 'oben-links', 'links']
['oben', 'oben', 'oben', 'oben-links', 'oben-links', 'links', 'oben-links']
['oben', 'oben', 'oben', 'oben', 'oben-links', 'oben-links', 'links']



In [58]:
aligned_v, aligned_w = backtrack_global_alignment(B_global, v, w, len(v), len(w))
print(f"Score: {score_global}")
print("LGT-Alignment:")
print(aligned_v)
print(aligned_w)

# Save to file
with open("alignment.txt", "w") as file:
    file.write(f"Score: {score_global}\n")
    file.write("LGT-Alignment:\n")
    file.write(aligned_v + "\n")
    file.write(aligned_w + "\n")

Score: 3
LGT-Alignment:
ATGCG-
ATCCGC


# Aufgabe 3